In [ ]:
### Load libraries ###

# interactive plotting
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

# plotting libraries
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
sns.set()
plt.style.use('ggplot')
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})

# Data management libraries
import itertools
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Machine learning libraries
from sklearn.decomposition import PCA, FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity
import sklearn.mixture as mixture
from scipy import linalg
import math

# Other
from mltools import unsupervised_tools as UT

# Import data

# Load data
eolica = pd.read_csv("eolica.csv")

df = eolica
numeric_inputs = ['Tmax','Tmin','Tmed','Rmed','Vmax','Pmed_00_24','Pmed_00_06','Pmed_06_12', 'Pmed_12_18','Pmed_18_24']
X = df[numeric_inputs]
Y = df[['gen']]

# Preprocessing the values to perform PCA
numeric_features = X.select_dtypes(include=['int64','float64']).columns.values.tolist()
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X=X)

## PCA -----------------------------------------------------------
pca = PCA(n_components=len(X.columns),) #num de variables
X_pca = pca.fit_transform(X_transformed)

exp_variance = pd.DataFrame(data=pca.explained_variance_ratio_, index = ['PC' + str(n_pca + 1) for n_pca in range(pca.n_components)], columns=['Exp_variance'])
exp_variance['cum_Exp_variance'] = exp_variance['Exp_variance'].cumsum()
exp_variance

## PCA -----------------------------------------------------------
'''
Con esto se ve que ncomponents tiene que ser 5 para que el 
95% de la varianza sea explicada

pca = PCA(n_components=X_transformed.shape[1],)
X_pca = pca.fit_transform(X_transformed)

sns.barplot(data=exp_variance, x=exp_variance.index, y='Exp_variance', color='gray')
sns.lineplot(data=exp_variance, x=exp_variance.index, y='cum_Exp_variance', color='blue', label='Cumulative Proportion')
plt.ylabel('Proportion of Variance Explained')
plt.xlabel('Principal Component')
plt.legend()
plt.show()
'''

## PCA -----------------------------------------------------------
pca = PCA(n_components=5,) #95% of variance
X_pca = pca.fit_transform(X_transformed)

# Print the PCs
# loadings = pd.DataFrame(pca.components_.T * np.sqrt(pca.explained_variance_), columns=['PC' + str(pca + 1) for pca in range(pca.n_components)], index=numeric_inputs)

# fig, axes = plt.subplots(4, 1, figsize=(16,9))
# PC = 0
# for ax in axes.ravel():
#     sns.barplot(data=loadings, x=loadings.index, y=loadings.columns.values.tolist()[PC], color='gray', ax=ax)
#     PC += 1

# df_pca = pd.DataFrame(X_pca, columns=['PC' + str(n_pca + 1) for n_pca in range(pca.n_components)])
# df_pca['gen'] = Y.values
# df_pca['ccaa'] = df['ccaa']

# sns.lmplot(x='PC1', y='PC2', data=df_pca, hue='ccaa', fit_reg=False, scatter_kws={'alpha':0.5})	
# plt.show()

## Use gaussian mixture if you want to specify number of kernels
lowest_bic = np.infty
bic = []
n_components_range = range(1, 13)
cv_types = ['spherical', 'tied', 'diag', 'full']
for cv_type in cv_types:
    for n_components in n_components_range:
        # Fit a Gaussian mixture with EM
        gmm = mixture.GaussianMixture(n_components=n_components,
                                covariance_type=cv_type)
        gmm.fit(X_pca)
        bic.append(gmm.bic(X_pca))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm

bic = np.array(bic)
color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue', 'darkorange', 'green', 'red', 'blue', 'gray', 'brown', 'purple', 'pink', 'yellow'])
gmm = mixture.GaussianMixture(n_components=6, covariance_type=cv_type)
gmm.fit(X_pca)
clf = gmm
bars = []

# Plot the BIC scores
plt.figure(figsize=(8, 6))
spl = plt.subplot(2, 1, 1)
for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
    xpos = np.array(n_components_range) + .2 * (i - 2)
    bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                  (i + 1) * len(n_components_range)],
                        width=.2, color=color))
plt.xticks(n_components_range)
plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
plt.title('BIC score per model')
xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
    .2 * np.floor(bic.argmin() / len(n_components_range))
plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
spl.set_xlabel('Number of components')
spl.legend([b[0] for b in bars], cv_types)

# Plot the winner
splot = plt.subplot(2, 1, 2)
Y_ = clf.predict(X_pca)
for i, (mean, cov, color) in enumerate(zip(clf.means_, clf.covariances_,
                                           color_iter)):
    v, w = linalg.eigh(cov)
    if not np.any(Y_ == i):
        continue
    plt.scatter(X_pca[Y_ == i, 0], X_pca[Y_ == i, 1], .8, color=color)

    # Plot an ellipse to show the Gaussian component
    angle = np.arctan2(w[0][1], w[0][0])
    angle = 180. * angle / np.pi  # convert to degrees
    v = 2. * np.sqrt(2.) * np.sqrt(v)
    ell = mpl.patches.Ellipse(mean, width=v[0], height=v[1], angle=180. + angle, color=color)
    ell.set_clip_box(splot.bbox)
    ell.set_alpha(.5)
    splot.add_artist(ell)

plt.xticks(())
plt.yticks(())
plt.title('Selected GMM: full model, %i components' % clf.n_components)
plt.subplots_adjust(hspace=.35, bottom=.02)
plt.show()


# Definir el número de escenarios y el tamaño de cada escenario
scenario_size = 1000
scenario_samples, _ = clf.sample(scenario_size)
scenario_df = pd.DataFrame(scenario_samples, columns=[f'PC{i+1}' for i in range(scenario_samples.shape[1])])

x_scenarios = pca.inverse_transform(scenario_samples) # Inverse transform the scenarios
x_scenarios = pd.DataFrame(scaler.inverse_transform(x_scenarios), columns=numeric_inputs) # Inverse scale the scenarios